# Spin Qubit Pulse Sequences with the HDAWG

This notebook shows you how to use pulse sequences with an HDAWG to realize various experiments. While the sequences are mostly general, they are adapted to those typically used for spin qubits by adding the gate pulsing to control between Coulomb and spin blockade.

List of experiemnts
* Rabi - sweep length of burst
* Ramsey variant 1 - sweep delay with constant burst duration
* Ramsey variant 2 - sweep burst duration at constant delay
* Ramsey variant 3 - sweep phase of second burst and delay between bursts

## 0. General Imports

In [1]:
import numpy as np

from laboneq.simple import *

DO_EMULATION = True  # run in emulation mode by default

## 1. Device Setup
### 1.1 Calibration

In [2]:
def calibrate_devices(device_setup):
    device_setup.logical_signal_groups["q0"].logical_signals[
        "drive_line"
    ].calibration = SignalCalibration(
        oscillator=Oscillator(
            uid="drive_q0_osc", frequency=1e8, modulation_type=ModulationType.HARDWARE
        ),
        mixer_calibration=MixerCalibration(
            voltage_offsets=[0.0, 0.0],
            correction_matrix=[
                [1.0, 0.0],
                [0.0, 1.0],
            ],
        ),
    )

### 1.2 Create device setup

In [3]:
descriptor = """\
instruments:
  HDAWG:
  - address: DEV8431
    uid: device_hdawg
connections:
  device_hdawg:
    - iq_signal: q0/drive_line
      ports: [SIGOUTS/2, SIGOUTS/3]
    - rf_signal: q0/coulomb_line_1
      ports: [SIGOUTS/0]
    - rf_signal: q0/coulomb_line_2
      ports: [SIGOUTS/1]
"""

device_setup = DeviceSetup.from_descriptor(
    descriptor,
    server_host="your_ip_address",
    server_port=8004,
    setup_name="MySetup",
)
calibrate_devices(device_setup)

## 2. Rabi Experiment: sweep burst length

### 2.1 Define parameters for experiment

In [4]:
## constant definition
LEN_COULOMB_CYCLE = 200e-9
LEN_READOUT = 2 * LEN_COULOMB_CYCLE
X90_DURATION = 10e-9

START = 0
STOP = LEN_COULOMB_CYCLE / 2
STEPS = 5
NUM_REP = 2

In [5]:
## define length sweep parameter
length_sweep_parameter = LinearSweepParameter(
    uid="length_sweep", start=START, stop=STOP, count=STEPS
)

### 2.2 Experiment

In [6]:
# define pulse shapes
coulomb_pulse = pulse_library.const(
    uid="coulomb_pulse", length=LEN_COULOMB_CYCLE / 2, amplitude=0.5
)
coulomb_readout = pulse_library.const(
    uid="coulomb_readout", length=LEN_READOUT, amplitude=1
)
drive_pulse = pulse_library.const(
    uid="rabi_drive_pulse", length=X90_DURATION, amplitude=1
)

In [7]:
## Create Experiment
exp = Experiment(
    "Rabi with Coulomb pulse",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("coulomb_1"),
        ExperimentSignal("coulomb_2"),
    ],
)

# define experiment
with exp.acquire_loop_rt(
    uid=("shots"), count=NUM_REP, averaging_mode=AveragingMode.SEQUENTIAL
):
    with exp.sweep(parameter=length_sweep_parameter):
        with exp.section(
            uid=("empty"),
            length=LEN_COULOMB_CYCLE / 2,
            alignment=SectionAlignment.RIGHT,
        ):
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.5)
            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.5)
        with exp.section(
            uid=("manipulation"),
            length=LEN_COULOMB_CYCLE / 2,
            alignment=SectionAlignment.RIGHT,
        ):
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.75)
            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.75)
            exp.play(signal="drive", pulse=drive_pulse, length=length_sweep_parameter)
        with exp.section(uid="qubit_readout", length=LEN_READOUT):
            exp.play(signal="coulomb_1", pulse=coulomb_readout, amplitude=0.3)
            exp.play(signal="coulomb_2", pulse=coulomb_readout, amplitude=0.3)
        with exp.section(
            uid="outer_trigger",
            length=LEN_READOUT,
            trigger={"drive": {"state": 1}},
            alignment=SectionAlignment.RIGHT,
        ):
            with exp.section(
                uid="inner_trigger",
                length=LEN_READOUT - 100e-9,
                trigger={"drive": {"state": 2}},
            ):
                exp.reserve(signal="drive")

### 2.3 Signal mapping

In [8]:
# define signal maps for different qubits
map_q0 = {
    "drive": "/logical_signal_groups/q0/drive_line",
    "coulomb_1": "/logical_signal_groups/q0/coulomb_line_1",
    "coulomb_2": "/logical_signal_groups/q0/coulomb_line_2",
}

# calibration  for qubit 0
calib_q0 = Calibration()
calib_q0["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=500e6,
        modulation_type=ModulationType.HARDWARE,
    )
)

### 2.4 Session

In [9]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)

[2024.02.22 16:06:20.573] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /Users/james/Projects/PycharmProjects/SydneyQuantum/notebooks/laboneq_output/log
[2024.02.22 16:06:20.575] INFO    VERSION: laboneq 2.25.0
[2024.02.22 16:06:20.576] INFO    Connecting to data server at your_ip_address:8004
[2024.02.22 16:06:20.578] INFO    Connected to Zurich Instruments LabOne Data Server version 24.01 at your_ip_address:8004
[2024.02.22 16:06:20.579] WARNING HDAWG:dev8431: Include the device options 'HDAWG8/MF/ME/SKW/PC' in the device setup ('options' field of the 'instruments' list in the device setup descriptor). This will become a strict requirement in the future.
[2024.02.22 16:06:20.581] INFO    Configuring the device setup
[2024.02.22 16:06:20.583] INFO    The device setup is configured


In [10]:
if not session.connection_state.emulated:
    instrument_serial = device_setup.instrument_by_uid("device_hdawg").address
    device = session.devices[instrument_serial]
    device.triggers.out[2].delay(23.9e-9)

### 2.5 Run

In [11]:
session.run(exp)

[2024.02.22 16:06:20.598] INFO    Starting LabOne Q Compiler run...
[2024.02.22 16:06:20.605] INFO    Schedule completed
[2024.02.22 16:06:20.636] INFO     ────────────────────────────────────────────────────────────────── 
[2024.02.22 16:06:20.637] INFO      Device         AWG   SeqC LOC   CT entries   Waveforms   Samples  
[2024.02.22 16:06:20.638] INFO     ────────────────────────────────────────────────────────────────── 
[2024.02.22 16:06:20.639] INFO      device_hdawg     0         27            4           3      2880  
[2024.02.22 16:06:20.639] INFO      device_hdawg     1         68            5           4      1248  
[2024.02.22 16:06:20.640] INFO     ────────────────────────────────────────────────────────────────── 
[2024.02.22 16:06:20.641] INFO      TOTAL                      95            9                  4128  
[2024.02.22 16:06:20.642] INFO     ────────────────────────────────────────────────────────────────── 
[2024.02.22 16:06:20.647] INFO    Finished LabOne Q Com

Results(experiment=Experiment(uid='Rabi with Coulomb pulse', signals={'drive': ExperimentSignal(uid='drive', calibration=SignalCalibration(oscillator=Oscillator(uid='osc_0', frequency=500000000.0, modulation_type=ModulationType.HARDWARE, carrier_type=CarrierType.RF), local_oscillator=None, mixer_calibration=None, precompensation=None, port_delay=None, port_mode=None, delay_signal=None, voltage_offset=None, range=None, threshold=None, amplitude=None, amplifier_pump=None, added_outputs=None), mapped_logical_signal_path='/logical_signal_groups/q0/drive_line'), 'coulomb_1': ExperimentSignal(uid='coulomb_1', calibration=None, mapped_logical_signal_path='/logical_signal_groups/q0/coulomb_line_1'), 'coulomb_2': ExperimentSignal(uid='coulomb_2', calibration=None, mapped_logical_signal_path='/logical_signal_groups/q0/coulomb_line_2')}, version=DSLVersion.V3_0_0, epsilon=0.0, sections=[AcquireLoopRt(uid='shots', alignment=SectionAlignment.LEFT, execution_type=ExecutionType.REAL_TIME, length=None

### 2.6 View experiment in pulse sheet viewer

In [12]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Spin qubit length Rabi", compiled_exp)

[2024.02.22 16:06:20.711] INFO    Recompiling the experiment due to missing extra information in the compiled experiment. Compile with `OUTPUT_EXTRAS=True` and `MAX_EVENTS_TO_PUBLISH=1000` to bypass this step with a small impact on the compilation time.
[2024.02.22 16:06:20.716] INFO    Starting LabOne Q Compiler run...
[2024.02.22 16:06:20.722] INFO    Schedule completed
[2024.02.22 16:06:20.755] INFO    Finished LabOne Q Compiler run.
[2024.02.22 16:06:20.774] INFO    Writing html file to /Users/james/Projects/PycharmProjects/SydneyQuantum/notebooks/Spin qubit length Rabi_2024-02-22-16-06-20.html


/Users/james/Projects/PycharmProjects/SydneyQuantum/notebooks/Spin qubit length Rabi_2024-02-22-16-06-20.html

## 3. Ramsey Experiment I: sweep wait time at constant burst length
### 3.1 Define parameters for experiment

In [13]:
START = 0
STOP = LEN_COULOMB_CYCLE / 2 - 2 * X90_DURATION
STEPS = 3
NUM_REP = 20

In [14]:
## Define sweep parameter
sweep_delay = LinearSweepParameter(
    uid="Ramsey_delay", start=START, stop=STOP, count=STEPS
)

### 3.2 Experiment

In [15]:
## Create Experiment
exp = Experiment(
    "Ramsey variant I",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("coulomb_1"),
        ExperimentSignal("coulomb_2"),
    ],
)

# define experiment
with exp.acquire_loop_rt(
    uid="shots", count=NUM_REP, averaging_mode=AveragingMode.SEQUENTIAL
):
    with exp.sweep(uid="sweep", parameter=sweep_delay):
        with exp.section(
            uid="qubit_excitation",
            alignment=SectionAlignment.RIGHT,
            length=LEN_COULOMB_CYCLE,
        ):
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.5)
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.75)

            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.5)
            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.75)

            exp.play(signal="drive", pulse=drive_pulse)
            exp.delay(signal="drive", time=sweep_delay)
            exp.play(signal="drive", pulse=drive_pulse)
        with exp.section(uid="qubit_readout", length=LEN_READOUT):
            with exp.section(uid="readout_pulses"):
                exp.play(signal="coulomb_1", pulse=coulomb_readout, amplitude=0.3)
                exp.play(signal="coulomb_2", pulse=coulomb_readout, amplitude=0.3)
            with exp.section(
                uid="outer_trigger",
                length=LEN_READOUT,
                trigger={"drive": {"state": 1}},
                alignment=SectionAlignment.RIGHT,
            ):
                with exp.section(
                    uid="inner_trigger",
                    length=LEN_READOUT - 100e-9,
                    trigger={"drive": {"state": 2}},
                ):
                    exp.reserve(signal="drive")

### 3.3 Signal mapping

In [16]:
# define signal maps for different qubits
map_q0 = {
    "drive": "/logical_signal_groups/q0/drive_line",
    "coulomb_1": "/logical_signal_groups/q0/coulomb_line_1",
    "coulomb_2": "/logical_signal_groups/q0/coulomb_line_2",
}

# calibration  for qubit 0
calib_q0 = Calibration()
calib_q0["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=500e6,
        modulation_type=ModulationType.HARDWARE,
    )
)

### 3.4 Connect Session and Run

In [17]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)

session.run(exp)

[2024.02.22 16:06:20.830] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /Users/james/Projects/PycharmProjects/SydneyQuantum/notebooks/laboneq_output/log
[2024.02.22 16:06:20.831] INFO    VERSION: laboneq 2.25.0
[2024.02.22 16:06:20.833] INFO    Connecting to data server at your_ip_address:8004
[2024.02.22 16:06:20.834] INFO    Connected to Zurich Instruments LabOne Data Server version 24.01 at your_ip_address:8004
[2024.02.22 16:06:20.835] WARNING HDAWG:dev8431: Include the device options 'HDAWG8/MF/ME/SKW/PC' in the device setup ('options' field of the 'instruments' list in the device setup descriptor). This will become a strict requirement in the future.
[2024.02.22 16:06:20.837] INFO    Configuring the device setup
[2024.02.22 16:06:20.838] INFO    The device setup is configured
[2024.02.22 16:06:20.843] INFO    Starting LabOne Q Compiler run...
[2024.02.22 16:06:20.847] INFO    Schedule completed
[2024.02.22 16:06:20.876] INFO     ───

Results(experiment=Experiment(uid='Ramsey variant I', signals={'drive': ExperimentSignal(uid='drive', calibration=SignalCalibration(oscillator=Oscillator(uid='osc_1', frequency=500000000.0, modulation_type=ModulationType.HARDWARE, carrier_type=CarrierType.RF), local_oscillator=None, mixer_calibration=None, precompensation=None, port_delay=None, port_mode=None, delay_signal=None, voltage_offset=None, range=None, threshold=None, amplitude=None, amplifier_pump=None, added_outputs=None), mapped_logical_signal_path='/logical_signal_groups/q0/drive_line'), 'coulomb_1': ExperimentSignal(uid='coulomb_1', calibration=None, mapped_logical_signal_path='/logical_signal_groups/q0/coulomb_line_1'), 'coulomb_2': ExperimentSignal(uid='coulomb_2', calibration=None, mapped_logical_signal_path='/logical_signal_groups/q0/coulomb_line_2')}, version=DSLVersion.V3_0_0, epsilon=0.0, sections=[AcquireLoopRt(uid='shots', alignment=SectionAlignment.LEFT, execution_type=ExecutionType.REAL_TIME, length=None, play_

### 3.5 View experiment in pulse sheet viewer

In [18]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Ramsey variant I", compiled_exp)

[2024.02.22 16:06:20.931] INFO    Recompiling the experiment due to missing extra information in the compiled experiment. Compile with `OUTPUT_EXTRAS=True` and `MAX_EVENTS_TO_PUBLISH=1000` to bypass this step with a small impact on the compilation time.
[2024.02.22 16:06:20.935] INFO    Starting LabOne Q Compiler run...
[2024.02.22 16:06:20.940] INFO    Schedule completed
[2024.02.22 16:06:20.968] INFO    Finished LabOne Q Compiler run.
[2024.02.22 16:06:20.993] INFO    Writing html file to /Users/james/Projects/PycharmProjects/SydneyQuantum/notebooks/Ramsey variant I_2024-02-22-16-06-20.html


/Users/james/Projects/PycharmProjects/SydneyQuantum/notebooks/Ramsey variant I_2024-02-22-16-06-20.html

## 4. Ramsey Experiment II: sweep burst length at constant wait time
### 4.1 Define parameters for experiment

In [19]:
# define constant delay
T_DELAY = 40e-9

In [20]:
## parameters for experiment
START = 0
STOP = (LEN_COULOMB_CYCLE / 2 - T_DELAY) / 2
STEPS = 5
NUM_REP = 2

### 4.2 Experiment

In [21]:
## Define sweep parameter
sweep_length = LinearSweepParameter(
    uid="pulse_length_sweep", start=START, stop=STOP, count=STEPS
)

In [22]:
## Create Experiment
exp = Experiment(
    "Ramsey variant II",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("coulomb_1"),
        ExperimentSignal("coulomb_2"),
    ],
)

# define experiment
with exp.acquire_loop_rt(
    uid="shots", count=NUM_REP, averaging_mode=AveragingMode.SEQUENTIAL
):
    with exp.sweep(uid="length_sweep", parameter=sweep_length):
        with exp.section(uid="qubit_excitation", alignment=SectionAlignment.RIGHT):
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.5)
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.75)

            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.5)
            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.75)

            exp.play(signal="drive", pulse=drive_pulse, length=sweep_length)
            exp.delay(signal="drive", time=T_DELAY)
            exp.play(signal="drive", pulse=drive_pulse, length=sweep_length)
        with exp.section(
            uid="qubit_readout", alignment=SectionAlignment.RIGHT, length=LEN_READOUT
        ):
            with exp.section(uid="readout_pulses"):
                exp.play(signal="coulomb_1", pulse=coulomb_readout, amplitude=0.3)
                exp.play(signal="coulomb_2", pulse=coulomb_readout, amplitude=0.3)
            with exp.section(
                uid="outer_trigger",
                length=LEN_READOUT,
                trigger={"drive": {"state": 1}},
                alignment=SectionAlignment.RIGHT,
            ):
                with exp.section(
                    uid="inner_trigger",
                    length=LEN_READOUT - 100e-9,
                    trigger={"drive": {"state": 2}},
                ):
                    exp.reserve(signal="drive")

### 4.3 Signal Mapping

In [23]:
# define signal maps for different qubits
map_q0 = {
    "drive": "/logical_signal_groups/q0/drive_line",
    "coulomb_1": "/logical_signal_groups/q0/coulomb_line_1",
    "coulomb_2": "/logical_signal_groups/q0/coulomb_line_2",
}

# calibration  for qubit 0
calib_q0 = Calibration()
calib_q0["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=100e6,
        modulation_type=ModulationType.HARDWARE,
    )
)

### 4.4 Connect Session and Run

In [24]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)

session.run(exp)

[2024.02.22 16:06:21.065] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /Users/james/Projects/PycharmProjects/SydneyQuantum/notebooks/laboneq_output/log
[2024.02.22 16:06:21.066] INFO    VERSION: laboneq 2.25.0
[2024.02.22 16:06:21.067] INFO    Connecting to data server at your_ip_address:8004
[2024.02.22 16:06:21.069] INFO    Connected to Zurich Instruments LabOne Data Server version 24.01 at your_ip_address:8004
[2024.02.22 16:06:21.071] WARNING HDAWG:dev8431: Include the device options 'HDAWG8/MF/ME/SKW/PC' in the device setup ('options' field of the 'instruments' list in the device setup descriptor). This will become a strict requirement in the future.
[2024.02.22 16:06:21.073] INFO    Configuring the device setup
[2024.02.22 16:06:21.074] INFO    The device setup is configured
[2024.02.22 16:06:21.079] INFO    Starting LabOne Q Compiler run...
[2024.02.22 16:06:21.087] INFO    Schedule completed
[2024.02.22 16:06:21.128] INFO     ───

Results(experiment=Experiment(uid='Ramsey variant II', signals={'drive': ExperimentSignal(uid='drive', calibration=SignalCalibration(oscillator=Oscillator(uid='osc_2', frequency=100000000.0, modulation_type=ModulationType.HARDWARE, carrier_type=CarrierType.RF), local_oscillator=None, mixer_calibration=None, precompensation=None, port_delay=None, port_mode=None, delay_signal=None, voltage_offset=None, range=None, threshold=None, amplitude=None, amplifier_pump=None, added_outputs=None), mapped_logical_signal_path='/logical_signal_groups/q0/drive_line'), 'coulomb_1': ExperimentSignal(uid='coulomb_1', calibration=None, mapped_logical_signal_path='/logical_signal_groups/q0/coulomb_line_1'), 'coulomb_2': ExperimentSignal(uid='coulomb_2', calibration=None, mapped_logical_signal_path='/logical_signal_groups/q0/coulomb_line_2')}, version=DSLVersion.V3_0_0, epsilon=0.0, sections=[AcquireLoopRt(uid='shots', alignment=SectionAlignment.LEFT, execution_type=ExecutionType.REAL_TIME, length=None, play

### 4.5 View experiment in pulse sheet viewer

In [25]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Ramsey variant II", compiled_exp)

[2024.02.22 16:06:21.235] INFO    Recompiling the experiment due to missing extra information in the compiled experiment. Compile with `OUTPUT_EXTRAS=True` and `MAX_EVENTS_TO_PUBLISH=1000` to bypass this step with a small impact on the compilation time.
[2024.02.22 16:06:21.243] INFO    Starting LabOne Q Compiler run...
[2024.02.22 16:06:21.251] INFO    Schedule completed
[2024.02.22 16:06:21.293] INFO    Finished LabOne Q Compiler run.
[2024.02.22 16:06:21.310] INFO    Writing html file to /Users/james/Projects/PycharmProjects/SydneyQuantum/notebooks/Ramsey variant II_2024-02-22-16-06-21.html


/Users/james/Projects/PycharmProjects/SydneyQuantum/notebooks/Ramsey variant II_2024-02-22-16-06-21.html

## 5. Ramsey Experiment III: 2D sweep, wait time vs phase of second pulse
### 5.1 Define parameters for experiment

In [26]:
## constant definition
X90_DURATION = 10e-9  # [s]

START = 0
STOP = 2 * np.pi
STEPS = 5
NUM_REP = 2

START_DELAY = 0
STOP_DELAY = LEN_COULOMB_CYCLE / 2 - 2 * X90_DURATION
STEPS_DELAY = 3

In [27]:
## define phase sweep parameter
sweep_phase = LinearSweepParameter(
    uid="phase_sweep", start=START, stop=STOP, count=STEPS
)
sweep_delay = LinearSweepParameter(
    uid="Ramsey_delay", start=START_DELAY, stop=STOP_DELAY, count=STEPS_DELAY
)

print(sweep_phase.values / np.pi)

[0.  0.5 1.  1.5 2. ]


### 5.2 Experiment

In [28]:
## Create Experiment
exp = Experiment(
    "Ramsey variant I",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("coulomb_1"),
        ExperimentSignal("coulomb_2"),
    ],
)

# define experiment
with exp.acquire_loop_rt(
    uid="shots", count=NUM_REP, averaging_mode=AveragingMode.CYCLIC
):
    with exp.sweep(uid="sweep_delay", parameter=sweep_delay):
        with exp.sweep(uid="sweep_phase", parameter=sweep_phase):
            with exp.section(
                uid="qubit_excitation",
                alignment=SectionAlignment.RIGHT,
                length=LEN_COULOMB_CYCLE,
            ):
                exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.5)
                exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.75)

                exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.5)
                exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.75)

                exp.play(signal="drive", pulse=drive_pulse, set_oscillator_phase=0)
                exp.delay(signal="drive", time=sweep_delay)
                exp.play(
                    signal="drive",
                    pulse=drive_pulse,
                    increment_oscillator_phase=sweep_phase,
                )
            with exp.section(
                uid="qubit_readout",
                alignment=SectionAlignment.RIGHT,
                length=LEN_READOUT,
            ):
                exp.play(signal="coulomb_1", pulse=coulomb_readout, amplitude=0.3)
                exp.play(signal="coulomb_2", pulse=coulomb_readout, amplitude=0.3)
            with exp.section(
                uid="outer_trigger",
                length=LEN_READOUT,
                trigger={"drive": {"state": 1}},
                alignment=SectionAlignment.RIGHT,
            ):
                with exp.section(
                    uid="inner_trigger",
                    length=LEN_READOUT - 100e-9,
                    trigger={"drive": {"state": 2}},
                ):
                    exp.reserve(signal="drive")

### 5.3 Signal Mapping

In [29]:
# define signal maps for different qubits
map_q0 = {
    "drive": "/logical_signal_groups/q0/drive_line",
    "coulomb_1": "/logical_signal_groups/q0/coulomb_line_1",
    "coulomb_2": "/logical_signal_groups/q0/coulomb_line_2",
}

# calibration  for qubit 0
calib_q0 = Calibration()
calib_q0["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=0,
        modulation_type=ModulationType.SOFTWARE,
    )
)

print("Set modulation frequency to 0 Hz to better observe the phase sweep.")

Set modulation frequency to 0 Hz to better observe the phase sweep.


### 5.4 Connect Session and Run

In [30]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)

session.run(exp)

[2024.02.22 16:06:21.364] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /Users/james/Projects/PycharmProjects/SydneyQuantum/notebooks/laboneq_output/log
[2024.02.22 16:06:21.366] INFO    VERSION: laboneq 2.25.0
[2024.02.22 16:06:21.368] INFO    Connecting to data server at your_ip_address:8004
[2024.02.22 16:06:21.369] INFO    Connected to Zurich Instruments LabOne Data Server version 24.01 at your_ip_address:8004
[2024.02.22 16:06:21.371] WARNING HDAWG:dev8431: Include the device options 'HDAWG8/MF/ME/SKW/PC' in the device setup ('options' field of the 'instruments' list in the device setup descriptor). This will become a strict requirement in the future.
[2024.02.22 16:06:21.372] INFO    Configuring the device setup
[2024.02.22 16:06:21.374] INFO    The device setup is configured
[2024.02.22 16:06:21.377] INFO    Starting LabOne Q Compiler run...
[2024.02.22 16:06:21.387] INFO    Schedule completed
[2024.02.22 16:06:21.450] INFO     ───

Results(experiment=Experiment(uid='Ramsey variant I', signals={'drive': ExperimentSignal(uid='drive', calibration=SignalCalibration(oscillator=Oscillator(uid='osc_3', frequency=0, modulation_type=ModulationType.SOFTWARE, carrier_type=CarrierType.RF), local_oscillator=None, mixer_calibration=None, precompensation=None, port_delay=None, port_mode=None, delay_signal=None, voltage_offset=None, range=None, threshold=None, amplitude=None, amplifier_pump=None, added_outputs=None), mapped_logical_signal_path='/logical_signal_groups/q0/drive_line'), 'coulomb_1': ExperimentSignal(uid='coulomb_1', calibration=None, mapped_logical_signal_path='/logical_signal_groups/q0/coulomb_line_1'), 'coulomb_2': ExperimentSignal(uid='coulomb_2', calibration=None, mapped_logical_signal_path='/logical_signal_groups/q0/coulomb_line_2')}, version=DSLVersion.V3_0_0, epsilon=0.0, sections=[AcquireLoopRt(uid='shots', alignment=SectionAlignment.LEFT, execution_type=ExecutionType.REAL_TIME, length=None, play_after=None

### 5.5 View experiment in pulse sheet viewer

In [31]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Ramsey variant III", compiled_exp)

[2024.02.22 16:06:21.649] INFO    Recompiling the experiment due to missing extra information in the compiled experiment. Compile with `OUTPUT_EXTRAS=True` and `MAX_EVENTS_TO_PUBLISH=1000` to bypass this step with a small impact on the compilation time.
[2024.02.22 16:06:21.656] INFO    Starting LabOne Q Compiler run...
[2024.02.22 16:06:21.683] INFO    Schedule completed
[2024.02.22 16:06:21.760] INFO    Finished LabOne Q Compiler run.
[2024.02.22 16:06:21.785] INFO    Writing html file to /Users/james/Projects/PycharmProjects/SydneyQuantum/notebooks/Ramsey variant III_2024-02-22-16-06-21.html


/Users/james/Projects/PycharmProjects/SydneyQuantum/notebooks/Ramsey variant III_2024-02-22-16-06-21.html